## Session 3 - Generative AI 🖼️

### Intro 📝

I denne hands-on-session vil all prosessering skje on-prem, uten å koble til eksterne API-er. Dette er valgt for å demonstrere hvordan konfidensielle data kan håndteres trygt.

OpenAI's teknologier, som er state-of-the-art, gir høy ytelse men kommer også med usikkerheter. De er ikke transparente i hvordan data prosesseres, og det er en risiko for model drift, hvor modellens ytelse kan variere over tid. I tillegg kan APIer bli utdatert eller fjernet, noe som kan påvirke systemets stabilitet.

Denne notebook'en innholder ikke mangen oppgaver, så bruk gjerne *litt tid* på å se hvordan ting er gjort og ekspreminter med å endre parametere osv.

Denne notebook'en er blitt inspirert av kurset [Generative AI AI av DeepLearning.ai](https://www.deeplearning.ai/courses/generative-ai-with-llms/)

Kos dykk!

### Start 🔥

Før vi begynner, må vi sikre at nødvendige biblioteker er installert. Selv om de fleste sannsynligvis allerede er på plass, er det lurt å dobbeltsjekke for å unngå komplikasjoner senere.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import evaluate
import numpy as np
import pandas as pd
from rich import print


from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
from rouge_score import rouge_scorer

print("OK :thumbsup:")

Vi laster ned et datasett med dialoger, [DIALOGSum](https://huggingface.co/datasets/knkarthick/dialogsum).

DialogSum er et strot datasett for dialogoppsummering, som består av 13 460 dialoger (pluss 100 tilbakeholdte data for tema-generering) med tilhørende manuelt merkede oppsummeringer og temaer.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

In [ ]:
dataset

In [ ]:
example_indices = [40, 240]

dash_line = 50*'[bold black]-'

for i, index in enumerate(example_indices):
    print(dash_line)
    print(f"[bold magneta] Example {index}")
    print(dash_line)
    print("[bold green]  INPUT DIALOGUE:")
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print("[bold blue] :woman: BASELINE HUMAN SUMMARY:")
    print(dataset['test'][index]['summary'])
    print(dash_line)


In [ ]:
# Det finnes flere størrelser, vi skal bruke en "liten" en
model_name='google/flan-t5-base'

# Kan ta ~2-4 minutter hvis modeller har ikke lastet ned tidligere
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
sentence = "This is fun!"

tokenizer = AutoTokenizer.from_pretrained(model_name)

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )

print('[bold] :robot: ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('[bold] :baby: DECODED SENTENCE:')
print(sentence_decoded)

### Oppgave 1: Tokens vs word ID lookup

Skriv en setning der du bøyer verbet, feks "This is fun!" -> "This is funnier!". 
Sjekk den encoded sentence, ser du noen likheter med forrige tensor? 

Hvilken fordeler er det med denne approachen vs feks. word ID lookup? ({"fun": 1, "funnier": 2, osv.})

In [ ]:

# La oss teste modellen, vi velger to tekster og sjekker hvordan modellen oppsummere teksten
## Prøv gjerne å endre index på hvilken dialog du henter ut 🤓
example_indices = [40, 240]

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(f'[bold magneta] Example {index}')
    print(dash_line)
    print(f'[bold green] INPUT PROMPT:')
    print(f'{dialogue}')
    print(dash_line)
    print(f'[bold blue] :woman: BASELINE HUMAN SUMMARY:')
    print(summary)
    print(dash_line)
    print(f'[bold red] :robot: MODEL GENERATION - WITHOUT PROMPT ENGINEERING:')
    print(f"[italic] {output}")
    print()


### Oppgave

Hvordan utførte modellen oppgaven?

## Promt-engineering

### Zero-shooting

Nå skal vi gi litt mer instrukter til modellen hva den skal gjøre, men vi lar den svare på spørsmålet alene.

Vi lager en "promt" template og setter inn dialogen inn:
```python
prompt = f"""
Summarize the following conversation:

{dialogue}

Summary:
    """

```

ℹ️ I Python kan du bruke f-strenger for å formatere strenger enkelt. Du bruker et "f" foran strengen og setter variabler eller uttrykk i krøllparenteser {}. Du kan legge

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation:

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )

    print(f'[bold magneta] Example {index}')
    print(dash_line)
    print(f'[bold green] INPUT PROMPT:')
    print(f'{dialogue}')
    print(dash_line)
    print(f'[bold blue] :man: BASELINE HUMAN SUMMARY:')
    print(summary)
    print(dash_line)
    print(f'[bold red] :robot: MODEL GENERATION - ZERO SHOT:')
    print(f"[italic] {output}")
    print()

### Oppgave
Hvordan utførte modellen oppgaven?

### One-shot Learning

Nå skal vise modellen hvordan en oppgave er utført med spørsmålet + svar, og dermed be den løse neste oppgave 🤓


Her er vår promt denne gangen:
```python
prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
dialogue = dataset['test'][example_index_to_summarize]['dialogue']

prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
```


In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    """ Lag en promt basert på index """
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
        Dialogue:

        {dialogue}

        What was going on?
        {summary}


        """
            
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']
            
        prompt += f"""
        Dialogue:

        {dialogue}

        What was going on?
        """
        
    return prompt


In [ ]:

# Velger et eksempel med spørsmålet+svaret
example_indices_full = [40]

# Hvilken tekst den skal oppsummere
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


In [ ]:
# Kjører modellen
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)


print(f'[bold magneta] Example {example_index_to_summarize}')
print(dash_line)
print(f'[bold blue] BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print(f'[bold red] MODEL GENERATION - ONE SHOT:')
print(f"[italic] {output}")



### Few-shooting promt
Nå gir vi modellen et lite antall eksempler, slik at den forstår mye mer av oppgaven den skal løse 🤓

In [ ]:

# Velger X eksempler med spørsmål og svar 
example_indices_full = [40, 80]

# Velger oppgaven den skal løse
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)



print(f'[bold magneta] Example {example_index_to_summarize}')
print(dash_line)
print(f'[bold blue] BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print(f'[bold red] MODEL GENERATION - FEW SHOT:')
print(f"[italic] {output}")



### Oppgave
Prøv å legg inn flere eksempler og sjekk hva som skjer 😈

Hva er ulempen å legge inn mangen eksempler til modellen? Er det noen restriksjoner i selve modellen?


### Oppgave
Test forskjellige parametere på å generere tekst.

Her er noen default parameters/values:
```python

generation_config = GenerationConfig(max_new_tokens=50, 
                                     do_sample=True, 
                                     temperature=1, 
                                     top_k=50,
                                     repetition_penalty=1.0,
                                     length_penalty=1,
                                     top_p=1)
```

Sjekk i mer detaljer angående hvilken config som kan bli brukt [GenerationConfig](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig), se f. eks. på *Parameters for manipulation of the model output logits*.



In [ ]:
generation_config = GenerationConfig(max_new_tokens=50, 
                                     do_sample=True, 
                                     temperature=1, 
                                     top_k=50,
                                     repetition_penalty=1.0,
                                     length_penalty=1,
                                     top_p=1)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0], 
    skip_special_tokens=True
)


print(f'[bold magneta] Example {example_index_to_summarize}')
print(dash_line)
print(f'[bold blue] BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print(f'[bold red] MODEL GENERATION - FEW SHOT:')
print(f"[italic] {output}")


## Rouge Metric


For å verifisere kvaliteten på LLM-modellen sammenlignet med baseline, må vi bruke noen metrikker for å måle tekstkvaliteten.

To mye brukte metrikker innen NLP/LLM er [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric)) og [Bleu](https://en.wikipedia.org/wiki/BLEU). I dette tilfellet skal vi bare bruke Rouge.

Vi vil teste standardmetrikker for å måle [precision/recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall) i LLM-oppgaver, og sammenligne strenger ved hjelp av [unigram og bi-gram](https://www.analyticsvidhya.com/blog/2021/09/what-are-n-grams-and-how-to-implement-them-in-python/).

Det finnes flere varianter, men vi skal kun se på de "enkle" 🤓.

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3'], use_stemmer=True)

candidate_summary = "it is cold outside, so get a jacket"
reference_summary = "it is very cold outside, so get a warm jacket"

print(f"{candidate_summary} <--> {reference_summary}")
print(dash_line)

scores = scorer.score(reference_summary, candidate_summary)
for key in scores:
    print(f'[bold]{key}: {scores[key]}')

print(dash_line)

candidate_summary = "Squatch eats pizza"
reference_summary = "pizza eats Squatch"

print(f"{candidate_summary} <--> {reference_summary}")

scores = scorer.score(reference_summary, candidate_summary)
for key in scores:
    print(f'[bold]{key}: {scores[key]}')



# Oppgave

Legg inn [rougeL](https://en.wikipedia.org/wiki/ROUGE_(metric)), hvordan løser den *Squatch eats pizza* oppgaven?



# Siste oppgave

Nå skal vi måle hvordan Zero-Shot vs Few-Shot utfører oppgaven i storskala. For å begynne litt smått, tester vi ut 10 datasett:
```python
dialogues = dataset['test'][0:10]['dialogue']
```

Vi lagrer resultatet i en dataframe, slik at vi kan se på resultatet nærmere.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

zero_shot_summaries, few_shot_summaries = [], []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    zero_shot_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    zero_shot_text_output = tokenizer.decode(zero_shot_outputs[0], skip_special_tokens=True)
    zero_shot_summaries.append(zero_shot_text_output)


    # Vi velger to eksempler som skal inn i few-shot attempt vår
    # Vi velger de to siste i datasettet.
    example_indices_full = [-1]

    # Velger oppgaven den skal løse
    few_shot_prompt = make_prompt(example_indices_full, idx)


    input_ids = tokenizer(few_shot_prompt, return_tensors="pt").input_ids
    few_shot_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    few_shot_text_output = tokenizer.decode(few_shot_outputs[0], skip_special_tokens=True)
    few_shot_summaries.append(few_shot_text_output)

print("[bold red] DONE! :smiley:")


In [ ]:
zipped_summaries = list(zip(human_baseline_summaries, zero_shot_summaries, few_shot_summaries))
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'zero_shot_summaries', 'few_shot_summaries'])
df

In [ ]:
# Vi bruker en fanzy Evaluate pakke fra hugginface
rouge = evaluate.load('rouge')


zero_shot_results = rouge.compute(
    predictions=zero_shot_summaries,
    references=human_baseline_summaries[0:len(zero_shot_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

few_shot_results = rouge.compute(
    predictions=few_shot_summaries,
    references=human_baseline_summaries[0:len(few_shot_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('[bold] Zero-shot:')
for key, value in zero_shot_results.items():
    print(f"[bold magneta] {key}: {value:0.4f}")

print('[bold] Few-shot:')
for key, value in few_shot_results.items():
    print(f"[bold magneta] {key}: {value:0.4f}")


In [ ]:
print("[bold] Absolutt prosentvis forbedring av few-shot-attempt over HUMAN BASELINE :chart:")

improvement = (np.array(list(few_shot_results.values())) - np.array(list(zero_shot_results.values())))
for key, value in zip(few_shot_results.keys(), improvement):
    print(f'[bold magneta] {key}: {value*100:.2f}%')

# Oppgave

Vi prøvde egentlig **one-shot**, legg på noen flere eksempler i koden og se forbedringen 😎

# Ekstra oppgaver, hvis det er mer tid igjen

* Test flere eksempler, øk til f. eks. 20 stk
* Plot distribusjonen av en metrikk på zero/few-shot attempt. Hva ser du?
* Kjør en større modell, se oversikten her [flan-t5 - huggingface](https://huggingface.co/docs/transformers/model_doc/flan-t5)
    * ⚠️ **ADVARSEL** dette kommer til å bruke mye minne(!) 🤯


Det er mye dokumentasjon angående Flan-T5 modellene her på [huggingface](https://huggingface.co/docs/transformers/model_doc/t5).


### Dagens Python Biblotek

Hvis du kjeder deg å skrive output/print meldinger, bruk [rich](https://github.com/Textualize/rich) biblioteket! Alt blir kjekkere med <span style="color:red">emojis</span> og <span style="color:blue">farger</span>. 🤓 🔥